In [1]:
from src.agents.rag_agent import retriever

docs = retriever.invoke("Why did pankaj get rejected from his f1 visa application?")
print(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(id='93cfe12e-0020-416d-8589-55d488a31da6', metadata={'source': 'https://medium.com/i-am-writing/my-f1-visa-application-was-rejected-at-the-delhi-us-embassy-9e954639a7b3', 'title': 'My F1 Visa Application Was Rejected at the Delhi US Embassy | by Arts2Survive | I Am Writing | Jun, 2025 | Medium', 'description': 'A lot has happened in a short span of time, and there is so much I want to share. But let me begin with a bit of background. I traveled to the United States in July 2023 to start my flight training…', 'language': 'en'}, page_content='My F1 Visa Application Was Rejected at the Delhi US Embassy | by Arts2Survive | I Am Writing | Jun, 2025 | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inMastodonI Am Writing·Writing about everything and anything.Member-only storyMy F1 Visa Application Was Rejected at the Delhi US EmbassyArts2Survive6 min read·Jun 2, 2025--ShareSource: US EmbassyNon-Member Link.A lot has happened in a short span of time, and there is s

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langsmith import traceable

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=1)

# Add decorator so this function is traced in LangSmith
@traceable()
def rag_bot(question: str) -> dict:
    # LangChain retriever will be automatically traced
    docs = retriever.invoke(question)
    docs_string = " \n".join(doc.page_content for doc in docs)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

    Documents:
    {docs_string}"""

    # langchain ChatModel will be automatically traced
    ai_msg = llm.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )

    return {"answer": ai_msg.content, "documents": docs}

In [16]:
docs = rag_bot("why was pankaj rejected from his f1 visa application?")

In [6]:
docs

{'answer': "Pankaj's F1 visa application was rejected at the Delhi embassy. The provided documents do not explicitly state the reason for the rejection.",
 'documents': [Document(id='4b2e5fb1-43b5-47d3-87ca-0a85bcf41dc2', metadata={'source': 'https://medium.com/i-am-writing/my-f1-visa-application-was-rejected-at-the-delhi-us-embassy-9e954639a7b3', 'title': 'My F1 Visa Application Was Rejected at the Delhi US Embassy | by Arts2Survive | I Am Writing | Jun, 2025 | Medium', 'description': 'A lot has happened in a short span of time, and there is so much I want to share. But let me begin with a bit of background. I traveled to the United States in July 2023 to start my flight training…', 'language': 'en'}, page_content='My F1 Visa Application Was Rejected at the Delhi US Embassy | by Arts2Survive | I Am Writing | Jun, 2025 | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inMastodonI Am Writing·Writing about everything and anything.Member-only storyMy F1 Visa Application 

In [17]:
examples = [
  {
    "inputs": {"question": "What are the main components covered in the LangChain features article?"},
    "outputs": {"answer": "The article discusses building an end‑to‑end pipeline including agents, MCP, LangGraph ToolNode integration, agent evaluations, and CI/CD deployment."}
  },
  {
    "inputs": {"question": "Why is evaluation important in LangChain workflows?"},
    "outputs": {"answer": "Evaluation is important because it measures performance, identifies gaps, and enables improvement over time using datasets, target functions, and metrics."}
  },
  {
    "inputs": {"question": "What went wrong during the F‑1 visa interview in Delhi with Pankaj?"},
    "outputs": {"answer": "Pankaj experienced multiple issues that started in the morning, leading to the rejection of the visa application."}
  },
  {
    "inputs": {"question": "What was the outcome of the second F‑1 visa attempt?"},
    "outputs": {"answer": "On the second attempt, the applicant passed the interview and obtained the F‑1 visa successfully."}
  },
  {
    "inputs": {"question": "How did Pankaj prepare differently for the second visa?"},
    "outputs": {"answer": "Pankaj reflected on the first rejection, prepared thoroughly, and confidently answered standard interview questions, which led to approval."}
  },
  {
    "inputs": {"question": "What criticism did Pankaja receive about their career choices?"},
    "outputs": {"answer": "Pankaj was told he was 'not loyal to any one field' and might lack productivity by switching roles frequently."}
  },
  {
    "inputs": {"question": "How did Pankaj react to the loyalty criticism?"},
    "outputs": {"answer": "Pankaj reflected deeply on those words and thought about how they are perceived by others during different phases of life."}
  }
]

In [18]:
from langsmith import Client
from dotenv import load_dotenv
load_dotenv()


True

In [19]:

client = Client()
# Create the dataset and examples in LangSmith
dataset_name = "Arts2Survive Blogs Q&A"
try:
    dataset = client.create_dataset(dataset_name=dataset_name)
    client.create_examples(
        dataset_id=dataset.id,
        examples=examples
    )
    print("dataset created and examples added...")
except Exception as e:
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("dataset already exists. loaded it...")




dataset already exists. loaded it...


In [20]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0).with_structured_output(CorrectnessGrade, method="json_schema", strict=True)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
                    QUESTION: {inputs['question']}
                    GROUND TRUTH ANSWER: {reference_outputs['answer']}
                    STUDENT ANSWER: {outputs['answer']}"""

    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions}, 
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

In [21]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
relevance_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0).with_structured_output(RelevanceGrade, method="json_schema", strict=True)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [22]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM 
grounded_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0).with_structured_output(GroundedGrade, method="json_schema", strict=True)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([{"role": "system", "content": grounded_instructions}, {"role": "user", "content": answer}])
    return grade["grounded"]

In [23]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"

    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [24]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)


View the evaluation results for experiment: 'rag-doc-relevance-aaf89e1e' at:
https://smith.langchain.com/o/cb35aec6-2b3b-4044-bfca-43dbfad00211/datasets/7f16a176-3de3-4657-b196-9bcb6e371b1e/compare?selectedSessions=d7d14d63-2fae-4443-a55a-04f82ca2be68




In [26]:
experiment_results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,What are the main components covered in the La...,The LangChain features article covers agent bu...,[page_content='All LangChain Features at One P...,None,The article discusses building an end‑to‑end p...,True,True,True,True,1.032213,09a4a935-3de9-4050-b53b-25153d6432f3,e87b0406-ba1b-46dd-a1f9-5f4476a0dcfc
1,What went wrong during the F‑1 visa interview ...,The author's visa application was rejected at ...,[page_content='My F1 Visa Application Was Reje...,None,Pankaj experienced multiple issues that starte...,True,True,True,True,1.170695,11e9a8cc-896d-4c4b-a36b-5ff1da795e10,7d29dc70-01e2-461e-a8c2-2f188266f885
2,What criticism did Pankaja receive about their...,Pankaja's colleague criticized their lack of l...,[page_content='I’m not confused. I’m just livi...,None,Pankaj was told he was 'not loyal to any one f...,True,True,True,True,1.103791,16e0b080-3723-4bad-9fb0-462413f8404b,9da2ab3b-ad54-4253-af99-8a5eaa77dd43
3,Why is evaluation important in LangChain workf...,"Evaluation helps measure performance, identify...",[page_content='All LangChain Features at One P...,None,Evaluation is important because it measures pe...,True,True,True,True,0.937637,41ca5893-4fd3-4ea1-849b-e2f272b6f29d,57e91f5a-7662-466e-9cdb-19be9c6745cf
4,How did Pankaj react to the loyalty criticism?,Pankaj was initially taken aback by the critic...,[page_content='I’m not confused. I’m just livi...,None,Pankaj reflected deeply on those words and tho...,True,True,True,True,0.930002,596e105a-55d0-4c7e-b72a-aa74940e76cd,01ce9508-d7c4-4197-b9f7-07edf9686be0
5,What was the outcome of the second F‑1 visa at...,The author's second F1 visa attempt was approv...,[page_content='My F1 Visa Got Approved on 2nd ...,None,"On the second attempt, the applicant passed th...",True,True,True,True,0.898383,9da37869-3f1b-4bf0-be67-ac71cc8d77f6,e853c5ee-7203-4231-9242-8495ec3a38ba
6,How did Pankaj prepare differently for the sec...,The provided documents do not contain informat...,[page_content='My F1 Visa Got Approved on 2nd ...,None,"Pankaj reflected on the first rejection, prepa...",False,True,True,True,0.847008,e010b7e6-4faa-47dc-95d1-f8c9735b88c5,7aa9c07e-ffb0-4eb1-be22-d5ead993133a
